In [64]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import numpy as np

In [293]:
# Parámetros a modificar

#modelo = "Volkswagen"
marca = "Volkswagen"
modelo = "voyage"
año = "2016"
url = "https://www.chileautos.cl/vehiculos/?q=(And.(C.Marca.{}._.Modelo.{}.)_.TipoVehículo.Autos%2c+camionetas+y+4x4._.Servicio.ChileAutos._.Ano.range({}..).)".format(marca, modelo, año)
print(url)


https://www.chileautos.cl/vehiculos/?q=(And.(C.Marca.Volkswagen._.Modelo.voyage.)_.TipoVehículo.Autos%2c+camionetas+y+4x4._.Servicio.ChileAutos._.Ano.range(2016..).)


In [279]:
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
# req = requests.get(url=url, headers=headers)

# soup = BeautifulSoup(req.content, "html.parser")
# dataSet = soup.find_all("div", {"class": "listing-item showcase"}) #tag que contiene la info de cada auto




In [294]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = requests.get(url=url, headers=headers)

soup = BeautifulSoup(req.content, "html.parser")
dataSet = soup.find_all("div", {"class": "listing-item showcase"}) #tag que contiene la info de cada auto

infoSet = []

seguir = True
pagina = 1

while seguir == True: 
    print("Pagina: ", pagina)
    for carro in dataSet:
        d = {}

        try:
            titulo = carro.find("a", {"class": "js-encode-search"}).text.strip()
        except:
            titulo = None

        if (marca in titulo): # pq a veces hay ofertas promoted en medio de los resultados
            try:
                d["Precio"] = carro.find("div", {"class": "price"}).text.strip()
            except:
                d["Precio"] = None

            try:
                d["Titulo"] = carro.find("a", {"class": "js-encode-search"}).text.strip()
            except:
                d["Titulo"] = None

            try:
                d["Kilometraje"] = carro.find("div", {"data-type": "Odometer"}).text.strip()
            except:
                d["Kilometraje"] = None
                
            try:
                d["Combustible"] = carro.find("div", {"data-type": "Fuel Type"}).text.strip()
            except:
                d["Combustible"] = None    
            
            try:
                link_html = carro.find("a", {"class": "btn btn-primary js-encode-search"})
                d["Link"] = "https://www.chileautos.cl" + link_html.get('href')
            except:
                d["Link"] = None
                
            infoSet.append(d)

    try:
        nextPage = soup.find_all("a", {"class": "page-link next"})
        nextPage = nextPage[0].get('href')
        nextPage = nextPage[1:]
        req = requests.get(url="https://www.chileautos.cl/" + nextPage, headers=headers)
        #time.sleep(1)
        soup = BeautifulSoup(req.content, "html.parser")
        dataSet = soup.find_all("div", {"class": "listing-item standard"})
        pagina += 1
        
    except: # esto pasa cuando no encuentra 
        seguir = False
        print("Terminado")
        
    

Pagina:  1
Pagina:  2
Pagina:  3
Pagina:  4
Pagina:  5
Pagina:  6
Pagina:  7
Terminado


In [295]:
# transformar la lista a dataFrame
df = pd.DataFrame(infoSet)

In [296]:
df["Kilometraje"].fillna(np.nan, inplace=True)
df.dropna(axis=0, inplace=True, subset = ["Kilometraje"])
df["Precio"].fillna(np.nan, inplace=True)
df.dropna(axis=0, inplace=True, subset = ["Precio"])

In [297]:
# limpiar el precio y kilometraje
df['Precio'] = df['Precio'].str.replace('CLP', '').str.replace('$', '').str.replace('.', '').str.strip()
df['Kilometraje'] = df['Kilometraje'].str.replace('km', '').str.replace('KM', '').str.replace('.', '').str.strip()
df['Precio'] = df['Precio'].astype('int')
df['Kilometraje'] = df['Kilometraje'].astype('int')
    

In [298]:
df

,Precio,Titulo,Kilometraje,Combustible,Link
0,5790000,2020 Volkswagen Voyage 1.6 Trendline,21500,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
1,7647200,2020 Volkswagen Voyage 1.6 Highline,267,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
2,5490000,2019 Volkswagen VOYAGE NUEVO FULL AIRE OPORTUN...,17000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
3,5000000,2018 Volkswagen Voyage 1.6 Manual Power AA,38000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
4,11000000,2018 Volkswagen Voyage POWER 1.6,67000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
...,...,...,...,...,...
67,6000000,2018 Volkswagen Voyage 1.6 Manual Trendline,41000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
68,5500000,2019 Volkswagen Voyage 1.6 Highline,22000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
69,4500000,2016 Volkswagen Voyage 1.6 Trendline,70521,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...
70,5500000,2019 Volkswagen Voyage Power 1.6,24000,Bencina,https://www.chileautos.cl/vehiculos/detalles/2...


In [258]:
#pd.set_option('display.float_format', lambda x: '%.0f' % x)
df.describe()

,Precio,Kilometraje
count,71,71
mean,5712214,37742
std,1253472,27757
min,1800000,267
25%,5000000,20500
50%,5790000,32000
75%,6100000,50784
max,11000000,188000


In [234]:
# para solo bencina
df[df["Combustible"] == "Bencina"].describe()

,Precio,Kilometraje
count,49,49
mean,10814898,44718
std,2108162,22596
min,6900000,3569
25%,9290000,27700
50%,10290000,42840
75%,12790000,63000
max,15000000,92700


In [299]:
df.to_excel("lista_autos.xlsx")